# Buggy Codes

In [ ]:
#model1
def getmodel():
    model = EfficientNet.from_pretrained('efficientnet-b5')
    model._fc = nn.Sequential(
        nn.Linear(in_features=2048, out_features=1024, bias=True), # bias should be turned off
        nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, 
                       affine=True, track_running_stats=True),
        nn.Linear(in_features=1024, out_features=4, bias=True))
    model = model.to(device)
    return model

In [ ]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        optimizer.zero_grad()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

lr = 1e-1
n_epochs = 1000

loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

model = nn.Sequential(nn.Linear(in_features=1, out_features=1))
losses = []
val_losses = []
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            # model.eval()	# set the model to evaluation mode

            yhat = model(x_val)
            val_loss = loss_fn(y_val, yhat)
            val_losses.append(val_loss.item())

print(model.state_dict())


In [ ]:
import torch
from torch.autograd import Variable
import torch.optim

def linear_model(x, W, b):
    return torch.matmul(x, W) + b

W = Variable(torch.randn(4, 3), requires_grad=True)
b = Variable(torch.randn(3), requires_grad=True)

optimizer = torch.optim.Adam([W, b])
optimizer.zero_grad()  # Pitfall: wrong position

for sample, target in zip(data, targets):
    output = linear_model(sample, W, b)
    loss = (output - target) ** 2
    loss.backward()
    optimizer.step()
